In [1]:
import pickle
import pandas as pd
import sklearn
import numpy as np
import scanpy as sc
import scipy.sparse
import anndata
import matplotlib.pylab as plt
from sklearn.decomposition import PCA
import copy
import logging as logg
from sklearn import preprocessing
import os 
import warnings
import scATAcat
import seaborn as sns
import random as rn 

warnings.filterwarnings('ignore')

In [2]:
# set the seed for reproducibility
sd = 1234
np.random.seed(sd)
rn.seed(sd)
%env PYTHONHASHSEED=0


env: PYTHONHASHSEED=0


#### define necessary parameters

In [4]:
results_dir = "../../../results/Fig6-Granja_PBMC_scATAC/Fig6-apply_scATAcat"
output_dir = results_dir+"/outputs/"
figures_dir = results_dir+"/figures/"
data_dir = "../../../data/Granja2019/PBMC_D10T1/"

for dir in [figures_dir, output_dir]:
    if not os.path.exists(dir):
        os.makedirs(dir)


### 0 - Load scATAC-seq data

In [5]:
ENCODE_coverage_per_cell_df= pickle.load(open(data_dir + '04_ENCODE_coverage_by_cell_matrix/GSM4138893_scATAC_PBMC_D10T1_per_cell_encode_cCRE_matrix_sparse.pkl','rb'))


In [6]:
ENCODE_coverage_per_cell_df

<926535x2891 sparse matrix of type '<class 'numpy.float32'>'
	with 25063385 stored elements in Compressed Sparse Row format>

In [7]:
ENCODE_cCREs = pd.read_csv(data_dir +"02_ENCODE_cCRE_coverage/GSM4138893_scATAC_PBMC_D10T1_ENCODE_cCREs.csv", index_col=0)
ENCODE_cCREs.columns = ["cCREs"]
ENCODE_cCREs.index = ENCODE_cCREs['cCREs']
ENCODE_cCREs.head()

,cCREs
cCREs,
chr1_181251_181601,chr1_181251_181601
chr1_190865_191071,chr1_190865_191071
chr1_778562_778912,chr1_778562_778912
chr1_779086_779355,chr1_779086_779355
chr1_779727_780060,chr1_779727_780060


In [9]:
cell_IDs = pd.read_csv(data_dir +"03_cellIDs_and_annotations/cell_IDs.csv")
cell_IDs.columns = ["cell_IDs"]
cell_IDs.index = cell_IDs["cell_IDs"]
cell_IDs.head()

,cell_IDs
cell_IDs,
TCAGGGCAGGGCATTG-1,TCAGGGCAGGGCATTG-1
GTTCAAGCAGTTACAC-1,GTTCAAGCAGTTACAC-1
CATTCCGGTGATAACA-1,CATTCCGGTGATAACA-1
GCGTTGGTCCAGTTAG-1,GCGTTGGTCCAGTTAG-1
TCAAAGCCAAGCACTT-1,TCAAAGCCAAGCACTT-1


### 1 - initialize the AnnData object

In [10]:
sc_completeFeatures_adata = anndata.AnnData(ENCODE_coverage_per_cell_df.transpose(), var=ENCODE_cCREs, obs=cell_IDs)


In [11]:
sc_completeFeatures_adata

AnnData object with n_obs × n_vars = 2891 × 926535
    obs: 'cell_IDs'
    var: 'cCREs'

### 2 - add binary layer to AnnData

In [12]:
scATAcat.add_binary_layer(sc_completeFeatures_adata, binary_layer_key="binary")


KeyboardInterrupt



## 3- calculate & plot cell and feature statistics

In [ ]:
scATAcat.cell_feature_statistics(sc_completeFeatures_adata, binary_layer_key ='binary')

In [ ]:
scATAcat.plot_feature_statistics(sc_completeFeatures_adata, threshold=3, bins=50, color="lightgrey", save=True, save_dir = figures_dir +"/feature_statistics_plot.png")

In [ ]:
scATAcat.plot_cell_statistics(sc_completeFeatures_adata, threshold=1000, bins=50, color="lightgrey", save=True, save_dir = figures_dir + "/cell_statistics_plot.png")

## 4- filter the cells and features

In [ ]:
sc_completeFeatures_adata

In [ ]:
sc_filteredFeatures_adata = scATAcat.preproces_sc_matrix(sc_completeFeatures_adata,cell_cutoff=1000,cell_cutoff_max=80000, feature_cutoff=3, remove_chrY = True, var_key = 'cCREs', copy=True)

In [ ]:
sc_filteredFeatures_adata

### 5- load & preprocess the bulk data

In [ ]:
bulk_by_ENCODE_peaks_df_annotated =  pickle.load(open( "../../../../scATAcat_notebooks_for_paper/jan2024/data/Corces2016_bulk_ATAC/02_ENCODE_coverage_by_prototypes_matrix/Corces2016bulkATAC_ENCODE_cCRE_overlappingPeaks_annotated.pkl", 'rb'))
bulk_by_ENCODE_peaks_df_annotated = bulk_by_ENCODE_peaks_df_annotated.reindex(sorted(bulk_by_ENCODE_peaks_df_annotated.columns), axis=1)

In [ ]:
bulk_by_ENCODE_peaks_df_annotated.columns

In [ ]:
#subset for terminal cell states
bulk_by_ENCODE_peaks_df_annotated
bulk_by_ENCODE_peaks_df_annotated= bulk_by_ENCODE_peaks_df_annotated[['Bcell_1', 'Bcell_2', 'Bcell_3', 'Bcell_4', 'CD4Tcell_1',
       'CD4Tcell_2', 'CD4Tcell_3', 'CD4Tcell_4', 'CD4Tcell_5',
       'CD8Tcell_1', 'CD8Tcell_2', 'CD8Tcell_3', 'CD8Tcell_4',
       'CD8Tcell_5', 'Mono_1', 'Mono_2', 'Mono_3', 'Mono_4', 'Mono_5',
       'Mono_6', 'NKcell_1', 'NKcell_2', 'NKcell_3', 'NKcell_4',
       'NKcell_5', 'NKcell_6']]

In [ ]:
# include pDCs
pDCs = pickle.load(open( '../../../../scATAcat_notebooks_for_paper/jan2024/data/Corces2016_bulk_ATAC/02_ENCODE_coverage_by_prototypes_matrix/Calderon2019_pDC_ENCODE_cCRE_overlappingPeaks_annotated.pkl', 'rb'))
pDCs.head()

In [ ]:
bulk_by_ENCODE_peaks_df_annotated = pd.concat([bulk_by_ENCODE_peaks_df_annotated, pDCs.loc[bulk_by_ENCODE_peaks_df_annotated.index,:]], axis=1 )

In [ ]:
bulk_by_ENCODE_peaks_df_annotated.head()

In [ ]:
bulk_completeFeatures_adata = scATAcat.generate_bulk_sparse_AnnData(bulk_by_ENCODE_peaks_df_annotated)

In [ ]:
bulk_completeFeatures_adata

In [ ]:
bulk_completeFeatures_adata = scATAcat.preprocess_bulk_adata(bulk_completeFeatures_adata, remove_chrY=True, var_key = 'cCREs', copy=False)

### 6 - Overlap bulk and sc features

- before we proceed with sc analysis, we need to overlap the features:
    - we don't want the feature that does not have any counts in bulk data to influence clustering
    - similarly, we want the projection to be defined only with the same feature set

In [ ]:
sc_bulk_common_vars = scATAcat.overlap_vars(sc_filteredFeatures_adata, bulk_completeFeatures_adata)

In [ ]:
len(sc_bulk_common_vars)

In [ ]:
sc_commonFeatures_adata = scATAcat.subset_adata_vars(sc_filteredFeatures_adata, vars_list=sc_bulk_common_vars, copy_=True)


In [ ]:
bulk_commonFeatures_adata = scATAcat.subset_adata_vars(bulk_completeFeatures_adata, vars_list=sc_bulk_common_vars, copy_=True)


### 7- doublet removal

In [ ]:
#results from AMLUET: these are the cells inferred to be doublets:
    
multiplet_cells = pd.read_csv(data_dir + '06_doublet_detection/MultipletCellIds_01.txt', header=None)    


In [ ]:
multiplet_cells

In [ ]:
non_multiplet_cells = list(set(sc_commonFeatures_adata.obs['cell_IDs']) - set(multiplet_cells[0]))

In [ ]:
sc_commonFeatures_adata = scATAcat.subset_adata_obs(sc_commonFeatures_adata, obs_list= non_multiplet_cells, copy_=False)

In [ ]:
sc_commonFeatures_adata

### 8- apply TF-IDF

In [ ]:
scATAcat.apply_TFIDF_sparse(sc_commonFeatures_adata, binary_layer_key='binary', TFIDF_key='TF_logIDF' )

### 9 - subset matrices to differential cCREs

In [ ]:
pairwise_top2000_cCREs = pd.read_table("../../../../scATAcat_notebooks_for_paper/jan2024/data/Corces2016_bulk_ATAC/03_get_differentially_accessible_regions/terminal_cell_states-for_PBMC_scmultiome/Corces2016_pairwise_differential_cCREs_FDR0.05_top2000_of_each_pair_sorted_exactCREs.csv",delimiter="\t",header=None)

pairwise_top2000_cCREs.head()

In [ ]:
len(pairwise_top2000_cCREs)

In [ ]:
common_differential_vars = list(set(list(sc_bulk_common_vars)) & set(list(pairwise_top2000_cCREs[0].tolist())))

len(common_differential_vars)

In [ ]:
bulk_commonDiffFeatures_adata = scATAcat.subset_adata_vars(bulk_commonFeatures_adata,
                                                 vars_list=common_differential_vars,
                                                 copy_=True)

In [ ]:
sc_commonDiffFeatures_adata = scATAcat.subset_adata_vars(sc_commonFeatures_adata,
                                                 vars_list=common_differential_vars,
                                                 copy_=True)

### 10- dimention reduction and clustering 

In [ ]:
sc_commonDiffFeatures_adata

In [ ]:
scATAcat.apply_PCA(sc_commonDiffFeatures_adata, layer_key ='TF_logIDF', svd_solver='arpack', random_state=0)


In [ ]:
with plt.rc_context():
    sc.pl.pca_variance_ratio(sc_commonDiffFeatures_adata, n_pcs=50, log=True, show=False)
    plt.savefig(figures_dir + "/pca_variance_ratio.pdf", bbox_inches="tight")

In [ ]:
sc.pp.neighbors(sc_commonDiffFeatures_adata, n_pcs = 50, n_neighbors = 30, random_state=0)


In [ ]:
seqDepth_PC1_plot = sns.jointplot(
    x=sc_commonDiffFeatures_adata.obsm['X_pca'][:,0],
    y=np.sqrt(sc_commonDiffFeatures_adata.obsm['num_feature_per_cell']),
    kind="kde",fill=True, 
)
plt.xlabel("1st PC")
plt.ylabel("num_feature_per_cell")


In [ ]:
## correlation
np.corrcoef(sc_commonDiffFeatures_adata.obsm['X_pca'][:,0],
    np.sqrt(sc_commonDiffFeatures_adata.obsm['num_feature_per_cell']))

### 11 -  apply UMAP & leiden clustering

In [ ]:
leiden_resolution=1.0
leiden_key="leiden_"+ str(leiden_resolution)

In [ ]:
sc.tl.umap(sc_commonDiffFeatures_adata, random_state=0)

In [ ]:
sc.tl.leiden(sc_commonDiffFeatures_adata, resolution=leiden_resolution,key_added=leiden_key, random_state=0)

In [ ]:
with plt.rc_context():
    sc.pl.umap(sc_commonDiffFeatures_adata, color=leiden_key, show=False,size=25 , add_outline=False, frameon=False, title="",palette="tab20")
    plt.savefig(figures_dir + "/"+ leiden_key+ ".pdf", bbox_inches="tight")

#### loook at sequencing depth

In [ ]:
sc_commonDiffFeatures_adata.obs['num_feature_per_cell_'] = sc_commonDiffFeatures_adata.obsm['num_feature_per_cell']

In [ ]:
with plt.rc_context():  
    sc.pl.umap(sc_commonDiffFeatures_adata, color='num_feature_per_cell_', add_outline=False, frameon=False,title ="", save=False, size=25 )
    plt.savefig(figures_dir + "/seq_depth_umap.pdf", bbox_inches="tight")

### 12 - from pseudobulks according to the cluster assignments


In [ ]:
cell_cluster_assignments = pd.DataFrame(sc_commonDiffFeatures_adata.obs[leiden_key].copy())
cell_cluster_assignments


In [ ]:
cell_cluster_sizes = pd.DataFrame(cell_cluster_assignments[leiden_key].value_counts())
cell_cluster_sizes['leiden_clusters'] = cell_cluster_sizes.index
cell_cluster_sizes.head()

In [ ]:
for clust_id in set(sc_commonDiffFeatures_adata.obs[leiden_key].values):
    clust_df= sc_commonDiffFeatures_adata[sc_commonDiffFeatures_adata.obs[leiden_key]==clust_id]

In [ ]:
cell_types = ([(r.split('_')[0]) for r in clust_df.obs[leiden_key].index])

In [ ]:
# plot a bar chart
sns.set_style("whitegrid")
ax= sns.barplot(
    y="count", 
    x="leiden_clusters", 
    data=cell_cluster_sizes, 
    color='lightgrey');
ax.yaxis.grid(True,color="lightgrey")
ax.axes.set_xlabel("Leiden cluster ID")
ax.axes.set_ylabel("cluster size")

plt.savefig(figures_dir + "/cluster_sizes_"+leiden_key+".pdf", dpi=250)

In [ ]:
pseudobulk_commonFeatures_adata = scATAcat.generate_bulk_sparse_AnnData(scATAcat.get_pseudobulk_matrix_ext(adata_to_subset=sc_commonFeatures_adata, adata_to_get_clusters=sc_commonDiffFeatures_adata, cluster_key=leiden_key  ,method = 'sum'))

In [ ]:
scATAcat.preprocessing_libsize_norm_log2(pseudobulk_commonFeatures_adata)

In [ ]:
scATAcat.preprocessing_libsize_norm_log2(bulk_commonFeatures_adata)

### 13 - Projection


#### processing 

In [ ]:
bulk_commonDiffFeatures_adata = scATAcat.subset_adata_vars(bulk_commonFeatures_adata,
                                                 vars_list=common_differential_vars,
                                                 copy_=True)

In [ ]:
bulk_commonDiffFeatures_adata

In [ ]:
pseudobulk_commonDiffFeatures_adata = scATAcat.subset_adata_vars(pseudobulk_commonFeatures_adata,
                                                 vars_list=common_differential_vars,
                                                 copy_=True)

In [ ]:
scATAcat.preprocessing_standardization(bulk_commonDiffFeatures_adata, input_layer_key="libsize_norm_log2", zero_center=True)

In [ ]:
bulk_commonDiffFeatures_adata

In [ ]:
scATAcat.preprocessing_standardization(pseudobulk_commonDiffFeatures_adata, input_layer_key="libsize_norm_log2", zero_center=False,
                              output_layer_key= "libsize_norm_log2_bulk_scaled_diff",
                              std_key= None,  mean_key=None,
                              std_ = bulk_commonDiffFeatures_adata.var["feature_std"],
                              mean_= bulk_commonDiffFeatures_adata.var["feature_mean"])

In [ ]:
## as an option, I can add the color codes from the clustering/ sc adata as a paramater for the pseudobulk matrix 
leiden_color_key = leiden_key+"_colors"
pseudobulk_commonDiffFeatures_adata.uns[leiden_color_key] = sc_commonDiffFeatures_adata.uns[leiden_color_key]

In [ ]:
result= scATAcat.projection(prototype_adata=bulk_commonDiffFeatures_adata, pseudobulk_adata=pseudobulk_commonDiffFeatures_adata, prototype_layer_key = "libsize_norm_log2_std", pseudobulk_layer_key="libsize_norm_log2_bulk_scaled_diff", color_key=leiden_color_key, pseudobulk_label_font_size =13, prototype_label_font_size = 0, 
                            prototype_colors = ['#38184C', "#008F8C", "#82018F",  "#7ED9B7", "#275974", "#D46A00"], pseudobulk_colors =  None, pseudobulk_point_size=200, prototype_point_size=180, pseudobulk_point_alpha=0.8, prototype_point_alpha=0.7, cmap='twilight_shifted', prototype_legend = True, pseudobulk_legend = True, save_path = figures_dir + "projection.png")


In [ ]:
result_noLabel = scATAcat.projection(prototype_adata=bulk_commonDiffFeatures_adata, pseudobulk_adata=pseudobulk_commonDiffFeatures_adata, prototype_layer_key = "libsize_norm_log2_std", pseudobulk_layer_key="libsize_norm_log2_bulk_scaled_diff", color_key=leiden_color_key, pseudobulk_label_font_size =0, prototype_label_font_size =0, 
                            prototype_colors = ['#38184C', "#008F8C", "#82018F",  "#7ED9B7", "#275974", "#D46A00"], pseudobulk_colors =  None, pseudobulk_point_size=250, prototype_point_size=250, pseudobulk_point_alpha=0.8, prototype_point_alpha=0.7, cmap='twilight_shifted', prototype_legend = True, pseudobulk_legend = True, save_path = figures_dir + "projection_noLabel.png")


In [ ]:
result_noLabel_noLegend = scATAcat.projection(prototype_adata=bulk_commonDiffFeatures_adata, pseudobulk_adata=pseudobulk_commonDiffFeatures_adata, prototype_layer_key = "libsize_norm_log2_std", pseudobulk_layer_key="libsize_norm_log2_bulk_scaled_diff", color_key=leiden_color_key, pseudobulk_label_font_size =0, prototype_label_font_size =0, 
                            prototype_colors = ['#38184C', "#008F8C", "#82018F",  "#7ED9B7", "#275974", "#D46A00"], pseudobulk_colors =  None, pseudobulk_point_size=250, prototype_point_size=250, pseudobulk_point_alpha=0.8, prototype_point_alpha=0.7, cmap='twilight_shifted', prototype_legend = False, pseudobulk_legend = False, save_path = figures_dir + "projection_noLabel_noLegend.png")


### 14 - Assign annotations to clusters


In [ ]:
heatmap = scATAcat.plot_pca_dist_heatmap(result[1],result[2])

In [ ]:
centroid_heatmap =  scATAcat.plot_pca_dist_cent_heatmap(result[1],result[2])


In [ ]:
heatmap[0].savefig(figures_dir +"/heatmap.png") 
centroid_heatmap[0].savefig(figures_dir +"/centroid_heatmap.png") 


In [ ]:
scATAcat.get_closest_prototype_to_pseudobulk(centroid_heatmap[1])

In [ ]:
clusterID_prediction_dict = scATAcat.get_closest_prototype_to_pseudobulk(centroid_heatmap[1])
clusterID_prediction_dict

In [ ]:
perc_df

In [ ]:
perc_df = scATAcat.get_pseudobulk_to_prototype_distance(centroid_heatmap[1], pbulk_to_prototype=True)

In [ ]:
cluster_to_pseudobulk_heatmap_plot = sns.clustermap(scATAcat.get_pseudobulk_to_prototype_distance(centroid_heatmap[1], pbulk_to_prototype=True).T,yticklabels=True,xticklabels=True, cmap="Blues")

In [ ]:
cluster_to_pseudobulk_heatmap_plot.savefig(figures_dir+"/cluster_to_pseudobulk_heatmap_plot.png") 


In [ ]:
cluster_to_pseudobulk_heatmap_plot.savefig(figures_dir+"/cluster_to_pseudobulk_heatmap_plot.svg") 


### 15- Post hoc analysis

In [ ]:
cell_cluster_assignments_with_predictions = copy.deepcopy(cell_cluster_assignments)

#### match the clusterID with predicted cell type

In [ ]:
# convert the cell id column to character 
cell_cluster_assignments_with_predictions[leiden_key] = cell_cluster_assignments_with_predictions[leiden_key].astype(str)

In [ ]:
cell_cluster_assignments_with_predictions.head()

In [ ]:
# refortmat the dict suct that it mactes the clust ID col
clusterID_prediction_dict_edited = dict((''.join(filter(str.isdigit, key)), value) for (key, value) in clusterID_prediction_dict.items())
clusterID_prediction_dict_edited

In [ ]:
cell_cluster_assignments_with_predictions['scATAcat_annotation'] = cell_cluster_assignments_with_predictions[leiden_key].map(clusterID_prediction_dict_edited)

In [ ]:
cell_cluster_assignments_with_predictions['cell_IDs'] = cell_cluster_assignments_with_predictions.index

In [ ]:
cell_cluster_assignments_with_predictions.to_csv(output_dir +"/scATAcat_annotations.csv")

## export AnnData object

In [ ]:

with open(output_dir +'/sc_commonDiffFeatures_adata.pkl', 'wb') as f:
    pickle.dump(sc_commonDiffFeatures_adata, f)

In [ ]:
cell_cluster_assignments.to_csv(output_dir +"cell_cluster_assignments.csv")